To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

Features in the notebook:
1. Uses Maxime Labonne's [FineTome 100K](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset.
1. Convert ShareGPT to HuggingFace format via `standardize_sharegpt`
2. Train on Completions / Assistant only via `train_on_responses_only`
3. Unsloth now supports Torch 2.4, all TRL & Xformers versions & Python 3.12!

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


# Unsloth achieves up to 2x faster fine-tuning speeds compared to traditional methods, with a significant reduction in memory usage (up to 70%).
# This makes it suitable for environments with constrained computational resources, like Google Colab or low-end GPUs​

# Unsloth leverages LoRA (Low-Rank Adaptation), which modifies only a small fraction (1-10%) of a model's parameters during training,
# instead of fine-tuning the entire model. This drastically reduces the computational and memory requirements while achieving comparable performance.
# It allows models to adapt to domain-specific tasks without retraining the entire network, enabling faster iterations and greater flexibility.

# By supporting 4-bit quantization, Unsloth minimizes memory usage during training and inference.
# Quantization reduces the precision of the weights and activations, which reduces memory demands and accelerates computation while preserving accuracy​.
#     - the weights of the models use only 4-bits representation.

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9c81bace-ddc9-524a-31bb-1223220298ee)


* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
#FROM LECTURE:
# need to understand...
#    what does it mean to quantize the weights in the network:
#           - To reduce the precision of the weights from the common 16-bits or 32-bits to 4-bits.

#    what are some of the hyperparameters:
#           -

#    why did you choose a specific model:
#           - The 3B model is a balance between computational efficiency and performance.
#           - Larger models (e.g., 70B) can provide better accuracy on complex tasks but require significantly more hardware resources.
#           - A 3B model is suitable for general-purpose tasks with constrained computational budgets.
#           - It’s particularly effective for applications like instruction-following, summarization, and general-purpose reasoning tasks​.

#    and talk about the properties of the model:

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally! # Rotary Position Embeddings (RoPE) contextualize a token
dtype = None # None for auto detection (auto optimzed). Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.# 4 bit only update subset of parameters

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.


##### HYPERParameter ? why choose specific model?
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

##Loading the Pre-trained Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"   <-- check this, why choose this, can be found in HuggingFace
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_dir = "/content/drive/My Drive/Colab Notebooks/outputs"
logging_dir = "/content/drive/My Drive/Colab Notebooks/log"

!ls "/content/drive/My Drive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 log	       outputs
 lora_model   'Yet another copy of Llama-3.2 1B+3B Conversational + 2x faster finetuning.ipynb'
 lora_model2


In [ ]:
#checkpoint loading

from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally! # Rotary Position Embeddings (RoPE) contextualize a token
dtype = None # None for auto detection (auto optimzed). Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.# 4 bit only update subset of parameters

checkpoint_dir = "/content/drive/My Drive/Colab Notebooks/outputs/checkpoint-60-2"  # Example: Load checkpoint from step 10


# Load model and tokenizer for Llama (or similar model)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=checkpoint_dir,  # Use the directory of your saved checkpoint
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # other parameters as needed (e.g., token if using gated models)
)

==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
### explain the parameter  PEFT (Parameter Efficient Fine-Tuning)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128    # number of trainable parameters. higher improve perf, need more compute pwr
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",         # layers undergo LoRA, like the attention mechanism (q_proj, k_proj, v_proj) query, key, value, and output
                      "gate_proj", "up_proj", "down_proj",],          # and feedforward layers (up_proj, down_proj) other potential projection layer
    lora_alpha = 16, # Larger lora_alpha values may allow for more significant adjustments to the model’s weights. 16 is okay before overfitting risk
    lora_dropout = 0, # Supports any, but = 0 is optimized # Explanation: prevent overfitting by randomly setting some of the weights to zero during training.
                                                          # Set to 0 for specialized task. Non-zero values (e.g., 0.1) are useful when regularization is necessary.
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407, # same weight for each run
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos] #remains text form instead of tokenize ID
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train") ##load dataset

train_dataset = dataset.shuffle(seed=3407).select([i for i in range(int(0.8 * len(dataset)))])  # 80% for training
valid_dataset = dataset.shuffle(seed=3407).select([i for i in range(int(0.89 * len(dataset)), int(0.9 * len(dataset)))])  # 1% for validation
test_dataset = dataset.shuffle(seed=3407).select([i for i in range(int(0.99 * len(dataset)), len(dataset))])  # 1% for test

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(train_dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)
#train_dataset = standardize_sharegpt(train_dataset)
#train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

valid_dataset = standardize_sharegpt(valid_dataset)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True)

test_dataset = standardize_sharegpt(test_dataset)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [ ]:
valid_dataset[5]["conversations"]

[{'content': 'The graph of the parabola defined by the equation $y=(x-2)^2+3$ is rotated 180 degrees about its vertex, then shifted 3 units to the left, then shifted 2 units down. The resulting parabola has zeros at $x=a$ and $x=b$. What is $a+b$? ',
  'role': 'user'},
 {'content': '1. Start by understanding the original equation of the parabola $y=(x-2)^2+3$.\n2. Identify the vertex of the original parabola at $(2,3)$ and the axis of symmetry at $x=2$.\n3. Understand that rotating the parabola 180 degrees about its vertex flips it upside down but keeps the same vertex, resulting in the equation $y=-(x-2)^2+3$.\n4. Recognize that shifting the parabola 3 units to the left moves the vertex and the axis of symmetry 3 units to the left, changing the equation to $y=-(x+1)^2+3$.\n5. Understand that shifting the parabola 2 units down moves the vertex and the axis of symmetry 2 units down, resulting in the final equation $y=-(x+1)^2+1$.\n6. Find the zeros of the final parabola by setting $y=0$

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
# why the parameter important in finetuning

from trl import SFTTrainer #supervised fine tuning trainer class
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported



trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, #wrong but still ok
    eval_dataset=valid_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer), # sequence per conversation
    dataset_num_proc = 2, #2 parallel processing
    packing = False, # Can make training 5x faster for short sequences. #conversations have varied sequence length, so not ideal to packing
    args = TrainingArguments(
        per_device_train_batch_size = 4, #batch per forward/backward pass , affects train spd ### ADD IF POSSIBLE
        gradient_accumulation_steps = 8, #Accumulates gradients over multiple steps before performing a parameter update.
                                          # to simulate large batch reduces noise, better estimation, less overfitting (batch size * gradient steps)
        warmup_steps = 10, # This sets the number of steps during which the learning rate will gradually increase from zero to its initial value.
                          # it prevents the optimizer from making large updates when the model weights are still far from optimal
                          ##For more stable training, consider increasing this to a larger number if you observe instability early in training.
                          ## Check Gradient & Loss
        #num_train_epochs = 1, # Set this for 1 full training run. ##### TRAIN SEVERAL EPOCH FOR BETTER RESULT
        max_steps = 60,  #overrides epoch, number of training steps ## 1 step = 1 batch
                        # forward pass, predict-loss, back pass, update gradient-weight
        learning_rate = 3e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5, #logs will be printed or saved after every single step (batch) during training ## INCREASE TO SAVE TIME
        optim = "adamw_8bit", #AdamW is preferred over Adam in many deep learning applications because it improves the way weight decay (regularization)
                              #is applied, preventing the problem of regularization causing interference with the learning rate.
        weight_decay = 0.01, # need to be selective layer?
        lr_scheduler_type = "linear", #schedule means the learning rate will decrease linearly from its initial value to zero over the course of training.
        seed = 3407, #you may want to experiment with different seeds in separate runs to ensure your results are not biased by a particular seed.
        output_dir = output_dir,
        logging_dir = logging_dir,
        report_to = "none", # Use this for WandB etc
        eval_strategy = "steps",  # Specify evaluation strategy ('steps' or 'epoch')
        eval_steps = 60,  # Run evaluation every 100 steps (or set it based on your needs)
        save_steps = 60,  # Optionally save model checkpoints
        load_best_model_at_end=True,  # Keep the best model after evaluation
        metric_for_best_model="eval_loss", # or any other metric to select the best model
        # Optionally, you can set `save_total_limit` to limit the number of saved checkpoints
        #save_total_limit=3,
    ),
)

Map (num_proc=2):   0%|          | 0/80000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only #focus on improving assistant's replies only, cant adjust with input variaties ## rely on pretrained
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.eval_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the mechanism behind the movement of chromosomes during cell division and how are microtubules involved in this process?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe movement of chromosomes during cell division is a crucial process that ensures the accurate segregation of genetic material into the two daughter cells. This process is primarily driven by the mitotic spindle, a structure composed of microtubules and associated proteins. Microtubules are dynamic, filamentous cytoskeletal proteins made up of α- and β-tubulin subunits that play a central role in chromosome movement.\n\nThe mechanism behind the movement of chromosomes during cell division can be divided into several stages:\n\n1. Prophase: During prophase, the chromosomes condense and become visible under a microscope. 

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                      \n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller than p, they cannot be equal to p. Therefore, they must be less than p. \nBut since p is a prime number, it can only be divided by 1 and itself. Therefore, neither q nor r can divide p. \nNow, let's consider the number (p-2). Since p is greater than 2, we know that (p-2) is a positive integer. \nWe can express (p-2) as the sum of q and r: (p-2) = q + r. \nAdding 2 to both sides, we get: p = (q+2) + (r

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.725 GB of memory reserved.


In [ ]:
#BELOW TRAINING RESULT IS NOT GOOD , WHY? TRAINING LOSS ALMOST 1? train several epoch?

trainer_stats = trainer.train()

# initiates the training process for the model using the configurations and data provided to the trainer object.

'''
Forward Pass:
- Passes the input data through the model to calculate the predictions and the loss (e.g., cross-entropy loss for text generation).

Backward Pass:
- Computes the gradients for each trainable parameter by backpropagating the loss.

Gradient Updates:
- Applies the optimizer (e.g., AdamW) to update the model's weights using the gradients.
If gradient clipping or weight decay is enabled, those adjustments are applied during this step.

Learning Rate Scheduler:
- Updates the learning rate based on the schedule defined in the TrainingArguments (e.g., linear decay).

Logging:
- Tracks and logs metrics (e.g., loss, learning rate) at specified intervals (logging_steps).

Checkpointing:
- Saves the model and optimizer state to output_dir at regular intervals or at the end of an epoch, depending on configuration.

Repeat for Epochs:
- The above steps repeat for the number of epochs specified in TrainingArguments.


Return Value: trainer_stats
The output of trainer.train() (stored in trainer_stats) is an object containing statistics and metadata about the training process. Key details typically include:
- Training Loss:
   -The loss values computed at each step or epoch.
- Final Model State:
   -The state of the model after training (weights, biases, etc.).
- Number of Steps/Epochs:
   -Total steps or epochs completed during training.
- Time Taken:
   -Total time spent on training.

This data is useful for analyzing training performance, debugging, or resuming fine-tuning.

'''

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
60,0.755000,0.750838


"\nForward Pass:\n- Passes the input data through the model to calculate the predictions and the loss (e.g., cross-entropy loss for text generation).\n\nBackward Pass:\n- Computes the gradients for each trainable parameter by backpropagating the loss.\n\nGradient Updates:\n- Applies the optimizer (e.g., AdamW) to update the model's weights using the gradients.\nIf gradient clipping or weight decay is enabled, those adjustments are applied during this step.\n\nLearning Rate Scheduler:\n- Updates the learning rate based on the schedule defined in the TrainingArguments (e.g., linear decay).\n\nLogging:\n- Tracks and logs metrics (e.g., loss, learning rate) at specified intervals (logging_steps).\n\nCheckpointing:\n- Saves the model and optimizer state to output_dir at regular intervals or at the end of an epoch, depending on configuration.\n\nRepeat for Epochs:\n- The above steps repeat for the number of epochs specified in TrainingArguments.\n\n\nReturn Value: trainer_stats\nThe output o

In [ ]:
import pandas as pd
import json

with open("/content/drive/My Drive/Colab Notebooks/outputs/checkpoint-60/trainer_state.json", "r") as f:
    trainer_state = json.load(f)


# Assuming `trainer_state.get("log_history", [])` gives the list of logs
log_history = trainer_state.get("log_history", [])

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(log_history)

# Display the table
print(df)

    epoch  grad_norm  learning_rate    loss  step  eval_loss  eval_runtime  \
0   0.002   0.104142        0.00015  0.7589     5        NaN           NaN   
1   0.004   0.133389        0.00030  0.7366    10        NaN           NaN   
2   0.006   0.131972        0.00027  0.8021    15        NaN           NaN   
3   0.008   0.122190        0.00024  0.6797    20        NaN           NaN   
4   0.010   0.127652        0.00021  0.7380    25        NaN           NaN   
5   0.012   0.165411        0.00018  0.7808    30        NaN           NaN   
6   0.014   0.152018        0.00015  0.7696    35        NaN           NaN   
7   0.016   0.156994        0.00012  0.7726    40        NaN           NaN   
8   0.018   0.149083        0.00009  0.7528    45        NaN           NaN   
9   0.020   0.149982        0.00006  0.7273    50        NaN           NaN   
10  0.022   0.158094        0.00003  0.7409    55        NaN           NaN   
11  0.024   0.155620        0.00000  0.7550    60        NaN    

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally! # Rotary Position Embeddings (RoPE) contextualize a token
dtype = None # None for auto detection (auto optimzed). Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.# 4 bit only update subset of parameters

# Assume test_dataset is already loaded
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert to PyTorch tensors and ensure columns are correct
tokenized_dataset = tokenized_dataset.rename_column("score", "label")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader
dataloader = DataLoader(tokenized_dataset, batch_size=8)

# Evaluate the model
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy:.4f}")

==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


OutOfMemoryError: CUDA out of memory. Tried to allocate 264.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 255.06 MiB is free. Process 9495 has 14.49 GiB memory in use. Of the allocated memory 14.24 GiB is allocated by PyTorch, and 94.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2729.0575 seconds used for training.
45.48 minutes used for training.
Peak reserved memory = 14.516 GB.
Peak reserved memory for training = 11.791 GB.
Peak reserved memory % of max memory = 98.427 %.
Peak reserved memory for training % of max memory = 79.95 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference ¤Leverages optimization specific to the FastLanguageModel framework.

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda") # this means to run it on cuda, on gpu. On huggingface you will only have cpu, so it will go much slower.

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding numbers. The sequence is: 0, 1, 1, 2, 3, 5, 8, 13, and so on.\n0 is usually considered to be the first element of the']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
''' PURPOSE OF THIS BLOCK:
Configures and runs a conversational model to respond to a user’s message in real time.
Specifically:
- Prepares the model and tokenizer for optimized inference.
- Formats the input message into a chat-compatible format.
- Generates text based on the user’s prompt.
- Streams the response as it’s generated for a more dynamic experience.
'''


'''
Purpose: Configures the model for optimized inference using the FastLanguageModel framework. This enables native optimizations that can double the inference speed.
Why?:
- During inference, specific optimizations (e.g., kernel fusion, weight quantization) can reduce the computational overhead.
- Particularly useful for smaller models, where speed is critical.
Impact:
- Improves throughput during tasks like text generation without modifying model accuracy.
'''

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

'''
What Is TextStreamer?
- A utility that streams tokens (words or subwords) generated by the model in real time.
- Displays the model’s output incrementally as it generates tokens.
Why Use It?: Enhances interactivity by allowing users to see results as they are generated instead of waiting for the entire output.
'''
from transformers import TextStreamer

'''
Purpose:
- Configures the TextStreamer to display generated text interactively.
   - tokenizer: Decodes tokens into readable text.
   - skip_prompt=True: Ensures only the generated text is streamed, excluding the input prompt.
Why?: Improves the user experience by focusing only on the new response generated by the model.
'''

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

# Generates a response based on the input messages.
# Purpose: Generates a continuation for the input prompt and streams the output to the console as it is generated.
# Why?:
   # Real-time streaming improves interactivity and user engagement.
   # Parameters like temperature and min_p control the creativity and quality of the generated response.
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,    # input_ids=inputs: Uses the formatted, tokenized input prepared earlier.
                   use_cache = True, temperature = 1.5, min_p = 0.1)                      # streamer=text_streamer: Outputs the generated text incrementally in real-time.
                                                                                          # max_new_tokens=128: Limits the generated text to a maximum of 128 tokens.
                                                                                          # use_cache=True: Optimizes generation by caching intermediate computations.
                                                                                          # temperature=1.5: Increases randomness in the generated output (higher values produce more diverse results).
                                                                                          # min_p=0.1: Enforces a minimum token probability threshold, avoiding overly unlikely choices.

1, 13<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/lora_model2") # Local saving
tokenizer.save_pretrained("/content/drive/My Drive/Colab Notebooks/lora_model2")
model.push_to_hub("emeses/lab2_model", token = "") # Online saving
tokenizer.push_to_hub("emeses/lab2_model", token = "") # Online saving

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/emeses/lab2_model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/My Drive/Colab Notebooks/lora_model2"

!ls "/content/drive/My Drive/Colab Notebooks"

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
# Purpose: This block demonstrates how to reload the saved model and tokenizer for inference.
#if False:
    from unsloth import FastLanguageModel
    import torch

    #model_path = "emeses/lab2_model"

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower is a large steel lattice structure located in the heart of Paris, France. The tower is approximately 300 meters tall, with four distinct levels. The lowest level is 50 meters tall and serves as a viewing deck, while the second level is 175 meters tall and has a café and restaurant. The top level is 276 meters tall, making it the highest point in Paris. The tower was built as a symbol of French engineering and architecture for the 1889 World's Fair held in Paris. Visitors can take elevators or stairs to ascend the tower and experience breathtaking views of the city.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:

'''
This code saves and uploads a machine learning model in various quantization formats to either local storage or the Hugging Face Hub using the GGUF (GPTQ General Unified Format).
This format is designed for efficient model storage and inference, especially in low-resource environments like edge devices.
'''


'''
What It Does:
save_pretrained_gguf: Saves the model locally in the Q8_0 format (8-bit quantization) along with its tokenizer.
push_to_hub_gguf: Uploads the quantized model to the Hugging Face Hub for shared access.
Why?
- 8-bit Quantization (Q8_0) reduces the model size and memory usage without significantly affecting performance.
- It's a balance between speed and accuracy.
'''
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

'''
- f16 (16-bit precision) maintains higher numerical precision, which is ideal for models where accuracy is more critical than memory efficiency.
- Useful for high-performance environments like GPUs with FP16 support.
'''

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

'''
Why?:
- 4-bit Quantization significantly reduces model size and memory requirements, making it ideal for deployment on resource-constrained devices like mobile phones or edge devices.
- However, this can lead to a slight loss in accuracy.
'''

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

'''
What It Does:
- Uploads the model to the Hugging Face Hub in multiple quantization formats in one operation.
- Supported formats in this example: q4_k_m, q8_0, and q5_k_m.
- Requires an authentication token (token) to upload the model.
Why?:
- This is a time-saving approach if you need to make the model available in several quantized formats to support various deployment scenarios:
- q4_k_m: Very efficient for edge devices but less accurate.
- q8_0: Balanced performance and precision.
- q5_k_m: A compromise between q4_k_m and q8_0.
*An edge device = hardware that is located closer to the "edge" of a network, meaning near the end users or physical world, rather than centralized cloud servers or data centers.
 ex: laptop. These devices perform computing tasks locally.
'''

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>